# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [21]:
print(geoapify_key)

2a7c785332264ef08f4d849002f3f2d8


In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
# city_data_df.shape

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aripuana,-9.1667,-60.6333,301.46,49,73,1.29,BR,2024-10-14 19:37:28
1,1,egilsstadir,65.2653,-14.3948,273.10,93,75,4.63,IS,2024-10-14 19:38:59
2,2,pisco,-13.7000,-76.2167,291.18,82,0,6.69,PE,2024-10-14 19:39:00
3,3,hasaki,35.7333,140.8333,296.64,53,78,2.54,JP,2024-10-14 19:39:01
4,4,vwawa,-9.1167,32.9333,291.08,50,100,1.15,TZ,2024-10-14 19:39:03


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
# %%capture --no-display

# Configure the map plot
humidity_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Humidity"]
)
# Display the map
humidity_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
# dropping cities where the max temp is larger than 80 degress fahrenheit (or 299 Kelvin) and a place that isn't very cloudy
city_data_df = city_data_df[city_data_df['Max Temp'] <= 299]
city_data_df = city_data_df[city_data_df['Cloudiness'] <= 90 ]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,egilsstadir,65.2653,-14.3948,273.10,93,75,4.63,IS,2024-10-14 19:38:59
2,2,pisco,-13.7000,-76.2167,291.18,82,0,6.69,PE,2024-10-14 19:39:00
3,3,hasaki,35.7333,140.8333,296.64,53,78,2.54,JP,2024-10-14 19:39:01
5,5,urbano santos,-3.2078,-43.4036,297.84,85,79,2.44,BR,2024-10-14 19:36:40
6,6,verkhniy baskunchak,48.2256,46.7217,274.09,34,33,4.02,RU,2024-10-14 19:39:05
...,...,...,...,...,...,...,...,...,...,...
575,575,kani keli,-12.9536,45.1033,298.74,81,28,3.60,YT,2024-10-14 19:50:19
577,577,san benito,36.6166,-121.0844,290.10,64,0,1.80,US,2024-10-14 19:50:21
578,578,tymovskoye,50.8456,142.6550,286.81,73,29,5.06,RU,2024-10-14 19:50:22
581,581,arno,60.1167,18.6333,278.71,82,57,5.78,SE,2024-10-14 19:50:26


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df.drop(['City_ID','Cloudiness','Wind Speed','Date'], axis =1, inplace = True)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Country,Hotel Name
1,egilsstadir,65.2653,-14.3948,273.10,93,IS,
2,pisco,-13.7000,-76.2167,291.18,82,PE,
3,hasaki,35.7333,140.8333,296.64,53,JP,
5,urbano santos,-3.2078,-43.4036,297.84,85,BR,
6,verkhniy baskunchak,48.2256,46.7217,274.09,34,RU,
...,...,...,...,...,...,...,...
575,kani keli,-12.9536,45.1033,298.74,81,YT,
577,san benito,36.6166,-121.0844,290.10,64,US,
578,tymovskoye,50.8456,142.6550,286.81,73,RU,
581,arno,60.1167,18.6333,278.71,82,SE,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "category": "accommodation.hotel",
    "apiKey":geoapify_key,
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
egilsstadir - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
urbano santos - nearest hotel: No hotel found
verkhniy baskunchak - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
katsuura - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
# %%capture --no-display

# Configure the map plot

hotel_city = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["City", "Hotel Name"]
)
# Display the map
hotel_city


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)